# NHL 2017-2018 Season Team interactive graph using Bokeh. Source http://www.hockeyabstract.com/testimonials/nhl2017-18. Thank you very much to the work of all those involved at Hockey Abstract for their work and sharing of this feature rich dataset.  

In [7]:
import pandas as pd
import numpy as np
from math import pi
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv('nhl_2017_18_condensed.csv')

In [9]:
#Import the required libraries
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Select
from bokeh.layouts import widgetbox,row, column
from bokeh.palettes import viridis

Loading BokehJS ...

### The below cell walks through the steps we took to create an interactive graph using a cut down feature set from Hockey Abstract data. The data is stored in the file 'nhl_2017_18_condensed.csv'. We personally flattened the dataset from Hockey Abstract and manipulated the data into its current form. If you use this CSV file, or plot, please credit this source as well as those at Hockey Abstract. Run the cell below and explore the different relationships and visualizations it can create.

### Any constructive feedback or suggestions are welcome. Contact info is williamzdupree.com or willdupree90@gmail.

In [12]:
#This function will push the bokeh plot with updateds
def modify_doc(doc):
    
    #Create figure
    p2=figure(plot_height=600,plot_width=700)
    
    #define dictionary for drop down menu purposes
    abrev_bokeh_dict_1 = {'PlInf_Ht':['Height','Inches'],'PlInf_Wt':['Weight','Lbs.'],
                   'PlInf_Age':['Age','Years'],'PlInf_Seasons':['Seasons Played','Number'],'PrimStat_GP':['Games Played','Number'],
                    'PrimStat_G':['Goals Scored','Number'],'PrimStat_A':['Assists Made','Number'],
                    'PrimStat_+/-':['Plus-Minus','Number'],'PrimStat_PIM':['Penalties in Minutes','Minutes'],
                    'PrimStat_Shifts':['Shifts Played','Number'],'PrimStat_TOI':['Time On Ice','Seconds'],
                    'IndStat_iSF':['Shots on Goal','Number'],'PenStat_iPenT':['Penalties Taken','Number'],
                    'PenStat_iPenD':['Penalties Drawn','Number'],'CarStat_GP':['Career Total Games Played','Number'],
                   'CarStat_G':['Career Total Goals','Number'],'CarStat_A':['Career Total Assists','Number'],
                    'CarStat_+/-':['Career Total Plus-Minus','Number'],'CarStat_PIM':['Career Total Penalties in Minutes','Number'],
                    'CarStat_TOI':['Career Total Time on Ice','Minutes']}
    # Make the ColumnDataSource for main graph: source
    
    #Scale factor to visualize salary
    sal_max = df['ContInf_Salary'].max()

    # Convert df to a ColumnDataSource: source
    source = ColumnDataSource(data={
            'x'       : df['PrimStat_G'],
            'y'       : df['PrimStat_Shifts'],
            'salary'      : (df['ContInf_Salary']/sal_max)*25,
            'country'      : df['PlInf_Nat'],
            'player_name'      : df['PlInf_Name'],
        })
    
    ####################################
    #Make the widgets that will go into our interactive graph
         
    #It will be slightly confusing, but for user's ease we will flip flop our dictionary so that the drop down
    #menu makes better sense of our options to choose from
    non_abrev = [item[0] for item in abrev_bokeh_dict_1.values() ]
    is_abrev = [item for item in abrev_bokeh_dict_1.keys()]
    
    #Creat the second dictionary where we have reversed our key value pair. The reasoning will be so that a 
    #user can make sense of our drop down menu, then Bokeh will update the value from mapping to the dictionary. This
    #Value is a column of our dataframe. Finally, when updating we use the reverse mapping to call values from the dataframe
    #column name to get the plot label and axes label
    abrev_bokeh_dict_2 = dict(zip(non_abrev,is_abrev))
    
    
    #These lines define the x and y axis drop down choices
    x_select = Select(
        options=non_abrev,
        value='Shifts Played',
        title='X-Axis Stat'
    )
    
    y_select = Select(
        options=non_abrev,
        value='Goals Scored',
        title='Y-Axis Stat'
    )
    
    
    
    
    #######################################
    #We now must define callback functions for our varius widgets
    
    # Define the callback function: update_plot
    def update_plot(attr, old, new):
        
        #Get new values from drop down values
        x = abrev_bokeh_dict_2[x_select.value]
        y = abrev_bokeh_dict_2[y_select.value]
        
        
        # Read the current value off the dropdown: x and y. Update source data using dictionary, NOT ColumnSourceData
        new_data = {
            'x'       : df[x],
            'y'       : df[y],
            'salary'      : (df['ContInf_Salary']/sal_max)*15+5,
            'country'      : df['PlInf_Nat'],
            'player_name'      : df['PlInf_Name'],
        }
        # Assign new_data to source.data
        
        # Set the x-axis label
        p2.xaxis.axis_label =abrev_bokeh_dict_1[x][0]+' ('+abrev_bokeh_dict_1[x][1]+')'

        # Set the y-axis label
        p2.yaxis.axis_label = abrev_bokeh_dict_1[y][0]+' ('+abrev_bokeh_dict_1[y][1]+')'
        
        source.data = new_data
        
    
        
        
    
    ####################################################
    #We must now call all of our change values incase one of the widget value changes,
    #here we can update both the graph and axes so that whenever any widget changes our
    #plot updates
    
    # Attach the update_plot callback to the 'value' property of y_select
    y_select.on_change('value',update_plot)
    
    # Attach the update_plot callback to the 'value' property of agg_select
    x_select.on_change('value',update_plot)
    
    
    # Make a color mapper: color_mapper
    color_mapper = CategoricalColorMapper(factors=df['PlInf_Nat'].unique().tolist(),palette=viridis(18))

    # Add a circle glyph to the figure p
    p2.circle('x', 'y', source=source,color=dict(field='country', transform=color_mapper),legend='country',size='salary')
    
    # Set the x-axis label
    p2.xaxis.axis_label =abrev_bokeh_dict_1['PrimStat_G'][0]+' ('+abrev_bokeh_dict_1['PrimStat_G'][1]+')'

    # Set the y-axis label
    p2.yaxis.axis_label = abrev_bokeh_dict_1['PrimStat_Shifts'][0]+' ('+abrev_bokeh_dict_1['PrimStat_Shifts'][1]+')'
    
    #Create a title
    p2.title.text = 'NHL 2017-2018 Stat Correlation'
    
    # Set Label sizes
    p2.title.text_font_size = '15pt'
    
    p2.yaxis.major_label_text_font_size = '11pt'
    p2.xaxis.major_label_text_font_size = '11pt'

    p2.xaxis.axis_label_text_font_size = '12pt'
    p2.yaxis.axis_label_text_font_size = '12pt'

    # Create a HoverTool object: hover
    hover = HoverTool(tooltips=[('Name','@player_name')])

    # Add the HoverTool object to figure p
    p2.add_tools(hover)
    
    ####################
    #Set the layout for the plot so it is visually pleasing.
    layout = column(row(widgetbox(x_select,y_select),p2))
    
    #This current work around sets the legend outside of the figure
    new_legend = p2.legend[0]
    p2.add_layout(new_legend,'right')
    
    #Customize legend border
    p2.legend.border_line_width = 3
    p2.legend.border_line_color = "black"
    p2.legend.border_line_alpha = 0.5
    
    #Customize plot border
    p2.outline_line_width = 3
    p2.outline_line_color = "black"
    p2.outline_line_alpha = 0.5
    
    #Set the doc and then show the modified document, having an interactive graph
    doc.add_root(layout)
show(modify_doc)